This notebook aims to join all the data from the different sources. This includes rental data, population and income data, various distance data, etc.

First, we want to join the population data with the rental data. The population data is contained in 2 files, one contains the actual census data and the other file contains the forecasted populations.

In [3]:
import pandas as pd
population_df = pd.read_excel('../data/curated/population_data.xlsx', sheet_name='Table 2.2')

Simple preprocessing for the census data to prepare it for the join

In [4]:
row = population_df.iloc[7] 
null_columns = [idx for idx, val in enumerate(row) if pd.isnull(val)]
years = [2011, 2012, 2013, 2014,2015, 2016,2017,2018,2019,2020,2021,2022] 

df_first_row = population_df.iloc[5].copy()

# Step 1: Rename NaN entries in the first row
df_first_row[population_df.columns.get_loc('Unnamed: 1')] = 'suburb'

for i, year in enumerate(years):
    base_index = 2 + (i * 4)  # Start at 2 and increment by 4 for each year
    
    df_first_row[population_df.columns.get_loc(f"Unnamed: {base_index}")] = f"population_{year}"
    df_first_row[population_df.columns.get_loc(f"Unnamed: {base_index + 1}")] = f'Births_{year}'
    df_first_row[population_df.columns.get_loc(f'Unnamed: {base_index + 2}')] = f'Total_fertility_rate_{year}'


population_df.iloc[5] = df_first_row

# Step 2: Set the 5th row (index 5) as the header
population_df.columns = population_df.iloc[5]  # Use the 5th row as the new header
population_df = population_df.drop(5)  # Drop the row that has been used as the header

population_df = population_df.iloc[6:]
population_df = population_df.reset_index(drop = True)
population_df = population_df.dropna(axis = 1, how = 'all')
population_df = population_df.dropna()

# Rename the second column in the DataFrame
population_df.rename(columns={population_df.columns[0]: 'sa2_code'}, inplace=True)

# Display the resulting DataFrame
population_df.head(10)

/tmp/ipykernel_379668/2592622724.py:8: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  df_first_row[population_df.columns.get_loc('Unnamed: 1')] = 'suburb'
/tmp/ipykernel_379668/2592622724.py:13: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  df_first_row[population_df.columns.get_loc(f"Unnamed: {base_index}")] = f"population_{year}"
/tmp/ipykernel_379668/2592622724.py:14: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  df

5,sa2_code,suburb,population_2011,Births_2011,Total_fertility_rate_2011,population_2012,Births_2012,Total_fertility_rate_2012,population_2013,Births_2013,...,Total_fertility_rate_2019,population_2020,Births_2020,Total_fertility_rate_2020,population_2021,Births_2021,Total_fertility_rate_2021,population_2022,Births_2022,Total_fertility_rate_2022
0,206011106,Brunswick East,8966,102,np,9208,116,np,9870,105,...,0.87,13064,116,0.78,12964,143,0.77,13296,143,0.75
1,206011107,Brunswick West,13864,180,np,13963,199,np,14057,184,...,1.12,15010,169,1.07,14497,159,1.07,14684,134,0.98
2,206011109,Pascoe Vale South,9860,145,np,9954,135,np,10038,121,...,1.6,10836,101,1.42,10463,113,1.46,10413,104,1.41
3,206011495,Brunswick - North,11981,192,np,12254,173,np,12548,167,...,1.17,14124,142,1.07,13077,129,0.99,13254,129,0.92
4,206011496,Brunswick - South,12006,159,np,12402,153,np,12836,165,...,1.01,14235,155,0.97,13208,134,0.92,13364,117,0.84
5,206011497,Coburg - East,12236,181,np,12483,191,np,12655,163,...,1.3,13265,158,1.23,12675,144,1.18,12949,153,1.15
6,206011498,Coburg - West,13998,226,np,14057,243,np,14188,196,...,1.68,14523,172,1.55,14184,179,1.54,14102,146,1.41
7,20601,Brunswick - Coburg,82911,1185,np,84321,1210,np,86192,1101,...,1.18,95057,1013,1.09,91068,1001,1.07,92062,926,1
8,206021110,Alphington - Fairfield,8561,96,np,8706,118,np,8827,105,...,1.24,9538,90,1.14,9069,98,1.19,9156,94,1.14
9,206021112,Thornbury,18258,275,np,18484,316,np,18773,279,...,1.34,19791,236,1.25,19153,214,1.18,19177,224,1.12


In [5]:
# Load the CSV file into a DataFrame
csv_file_path = '../data/curated/forecasted_populations.csv'
population_forecast = pd.read_csv(csv_file_path)

In [6]:
import numpy as np

# Add columns for future population forecasts if they don't exist in population_df
for year in range(2023, 2028):
    population_col_name = f'population_{year}'
    if population_col_name not in population_df.columns:
        population_df[population_col_name] = np.nan  # Initialize with NaN

In [7]:
population_forecast.head()

,Unnamed: 0,Code,Suburb,2023_Forecast,2024_Forecast,2025_Forecast,2026_Forecast,2027_Forecast
0,0,206011106,Brunswick East,13690,14083,14477,14871,15264
1,1,206011107,Brunswick West,14684,14684,14684,14684,14684
2,2,206011109,Pascoe Vale South,10388,10375,10368,10364,10363
3,3,206011495,Brunswick - North,13254,13254,13254,13254,13254
4,4,206011496,Brunswick - South,13364,13364,13364,13364,13364


Add the forecasted populations to the original census dataframe so that the population data is complete for every SA2

In [8]:
# Ensure the 'Code' and 'sa2_code' columns are of the same type (string)
population_df['sa2_code'] = population_df['sa2_code'].astype(str)
population_forecast['Code'] = population_forecast['Code'].astype(str)

# Debugging: Track if matches are found
matches_found = 0

# Iterate through each row in the population_forecast DataFrame
for index, row in population_forecast.iterrows():
    code = row['Code']
    
    # Find the matching row in population_df based on 'sa2_code'
    matching_index = population_df[population_df['sa2_code'] == code].index
    
    # Debugging: Check if the match was successful
    if not matching_index.empty:
        print(f"Match found for Code {code} at index {matching_index[0]}")
        matches_found += 1
        
        # Add the forecast columns to the matched row in population_df
        for year in range(2023, 2028):  # Iterating through 2023 to 2027
            # Correctly reference the column name in population_forecast
            forecast_col_name = f'{year}_Forecast'
            
            # Update the population_df only if the forecast column exists in population_forecast
            if forecast_col_name in population_forecast.columns:
                population_df.at[matching_index[0], f'population_{year}'] = row[forecast_col_name]
    else:
        print(f"No match found for Code {code}")

# Display the resulting DataFrame and number of matches found
print(f"Total matches found: {matches_found}")

# Display the resulting DataFrame
population_df.head(50)

Match found for Code 206011106 at index 0
Match found for Code 206011107 at index 1
Match found for Code 206011109 at index 2
Match found for Code 206011495 at index 3
Match found for Code 206011496 at index 4
Match found for Code 206011497 at index 5
Match found for Code 206011498 at index 6
Match found for Code 20601 at index 7
Match found for Code 206021110 at index 8
Match found for Code 206021112 at index 9
Match found for Code 206021499 at index 10
Match found for Code 206021500 at index 11
Match found for Code 20602 at index 12
Match found for Code 206031113 at index 13
Match found for Code 206031115 at index 14
Match found for Code 206031116 at index 15
Match found for Code 206031501 at index 16
Match found for Code 206031502 at index 17
Match found for Code 20603 at index 18
Match found for Code 206041117 at index 19
Match found for Code 206041118 at index 20
Match found for Code 206041119 at index 21
Match found for Code 206041120 at index 22
Match found for Code 206041121 at

5,sa2_code,suburb,population_2011,Births_2011,Total_fertility_rate_2011,population_2012,Births_2012,Total_fertility_rate_2012,population_2013,Births_2013,...,Births_2021,Total_fertility_rate_2021,population_2022,Births_2022,Total_fertility_rate_2022,population_2023,population_2024,population_2025,population_2026,population_2027
0,206011106,Brunswick East,8966,102,np,9208,116,np,9870,105,...,143,0.77,13296,143,0.75,13690.0,14083.0,14477.0,14871.0,15264.0
1,206011107,Brunswick West,13864,180,np,13963,199,np,14057,184,...,159,1.07,14684,134,0.98,14684.0,14684.0,14684.0,14684.0,14684.0
2,206011109,Pascoe Vale South,9860,145,np,9954,135,np,10038,121,...,113,1.46,10413,104,1.41,10388.0,10375.0,10368.0,10364.0,10363.0
3,206011495,Brunswick - North,11981,192,np,12254,173,np,12548,167,...,129,0.99,13254,129,0.92,13254.0,13254.0,13254.0,13254.0,13254.0
4,206011496,Brunswick - South,12006,159,np,12402,153,np,12836,165,...,134,0.92,13364,117,0.84,13364.0,13364.0,13364.0,13364.0,13364.0
5,206011497,Coburg - East,12236,181,np,12483,191,np,12655,163,...,144,1.18,12949,153,1.15,12949.0,12949.0,12949.0,12949.0,12949.0
6,206011498,Coburg - West,13998,226,np,14057,243,np,14188,196,...,179,1.54,14102,146,1.41,14058.0,14035.0,14022.0,14015.0,14012.0
7,20601,Brunswick - Coburg,82911,1185,np,84321,1210,np,86192,1101,...,1001,1.07,92062,926,1,92894.0,93726.0,94558.0,95390.0,96222.0
8,206021110,Alphington - Fairfield,8561,96,np,8706,118,np,8827,105,...,98,1.19,9156,94,1.14,9156.0,9156.0,9156.0,9156.0,9156.0
9,206021112,Thornbury,18258,275,np,18484,316,np,18773,279,...,214,1.18,19177,224,1.12,19177.0,19177.0,19177.0,19177.0,19177.0


Load the preprocessed rental data

In [9]:
# File path to the parquet file
listings_file = '../data/curated/preprocessed_rent_data.parquet'

# Load the parquet file into a DataFrame
rental_listings = pd.read_parquet(listings_file)

In [10]:
print(len(rental_listings))

196908


Perform some more preprocessing on rental data before it is joined

In [11]:
import pandas as pd

# Filter out rows with year before 2011
rental_listings = rental_listings[rental_listings['year'] >= 2011]

# Drop the specified columns
rental_listings = rental_listings.drop(columns=['is_new_development', 'propertyId', 'address'])

# Convert 'is_furnished' from boolean to 0 and 1
rental_listings['is_furnished'] = rental_listings['is_furnished'].astype(int)

# Display the first few rows of the filtered DataFrame
print(rental_listings.head())


   state          suburb  bedrooms  bathrooms        propertyTypes  carspaces  \
67   vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
68   vic       Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   
69   vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
70   vic  Brunswick East       2.0        1.0  [apartmentUnitFlat]        1.0   
71   vic  East Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   

   date_listed   latitude   longitude  price  is_furnished  year  month  day  \
67  2011-01-15 -37.815016  144.965968  380.0             0  2011      1   15   
68  2011-01-17 -37.816257  144.953288  550.0             0  2011      1   17   
69  2011-01-17 -37.821294  144.955768  370.0             0  2011      1   17   
70  2011-01-31 -37.760855  144.961691  429.0             0  2011      1   31   
71  2011-02-14 -37.810074  144.985476  460.0             0  2011      2   14   

   SA2_CODE21  
67  206041503  


Merge the rental data with the population data. For now this is only for the training data which is why only data upto 2024 is joined.

In [12]:
# Generate a list of population columns from 2011 to 2024
years = list(range(2011, 2025))

# Melt the DataFrame to long format
population_long_df = population_df.melt(
    id_vars=['sa2_code', 'suburb'],
    value_vars=[f'population_{year}' for year in years],
    var_name='year',
    value_name='population'
)

# Extract the year from the column name
population_long_df['year'] = population_long_df['year'].str.extract('(\d{4})').astype(int)

In [13]:
# Join the rental listings with the population data
merged_df = pd.merge(rental_listings, population_long_df, how='left', left_on=['SA2_CODE21', 'year'], right_on=['sa2_code', 'year'])

# Drop columns that were used for merging but are no longer needed
merged_df = merged_df.drop(columns=['sa2_code'])

# Display the first few rows of the merged DataFrame
print(merged_df.head(20))

   state        suburb_x  bedrooms  bathrooms        propertyTypes  carspaces  \
0    vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
1    vic       Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   
2    vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
3    vic  Brunswick East       2.0        1.0  [apartmentUnitFlat]        1.0   
4    vic  East Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   
5    vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
6    vic        St Kilda       2.0        1.0  [apartmentUnitFlat]        1.0   
7    vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
8    vic  Brunswick East       2.0        1.0  [apartmentUnitFlat]        1.0   
9    vic  East Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   
10   vic       Brunswick       2.0        1.0  [apartmentUnitFlat]        1.0   
11   vic   St Kilda East    

Remove rows that have missing values as there should be extremely few of them

In [14]:
import pandas as pd

# Check for missing values in each column
missing_counts = merged_df.isna().sum()

# Display the number of missing values for each column
print("Missing values per column:")
print(missing_counts)
print("\n")

# Check if any rows have missing values in critical columns
critical_columns = ['population', 'suburb_x', 'year', 'latitude', 'longitude']  # Adjust this list as needed
missing_critical = merged_df[critical_columns].isna().any(axis=1)

# Display rows with missing values in critical columns
print("Rows with missing values in critical columns:")
print(merged_df[missing_critical])


Missing values per column:
state             0
suburb_x          0
bedrooms          0
bathrooms         0
propertyTypes     0
carspaces         0
date_listed       0
latitude          0
longitude         0
price             0
is_furnished      0
year              0
month             0
day               0
SA2_CODE21       13
suburb_y         13
population       13
dtype: int64


Rows with missing values in critical columns:
       state         suburb_x  bedrooms  bathrooms        propertyTypes  \
5249     vic    Glen Waverley       5.0        2.0              [house]   
5355     vic   Mount Waverley       1.0        1.0              [house]   
9395     vic        RESERVOIR       2.0        1.0  [apartmentUnitFlat]   
9396     vic           EPPING       4.0        2.0          [townhouse]   
34723    vic        BRUNSWICK       3.0        3.0          [townhouse]   
54316    vic        Southbank       2.0        1.0  [apartmentUnitFlat]   
75372    vic         BALLARAT       3.0        

In [15]:
critical_columns = ['population', 'suburb_x', 'year', 'latitude', 'longitude']

# Drop rows with missing values in these critical columns
cleaned_df = merged_df.dropna(subset=critical_columns)

# Display the shape of the DataFrame before and after dropping rows
print(f"Original DataFrame shape: {merged_df.shape}")
print(f"DataFrame shape after dropping rows with missing values: {cleaned_df.shape}")

Original DataFrame shape: (151815, 17)
DataFrame shape after dropping rows with missing values: (151802, 17)


Now, we move on to joining this new dataframe with the distances of the closest amenity to each listing.

In [17]:
# File path to the parquet file
distances_file = '../data/curated/preprocessed_rent_distances_data.parquet'

# Load the parquet file into a DataFrame
distances_data = pd.read_parquet(distances_file)

In [28]:
distances_data.count()

csv_latitude              147647
csv_longitude             147647
distance_to_bus_stop      147647
distance_to_train_stop    147647
distance_to_tram_stop     147647
dtype: int64

In [30]:
# Check for uniqueness in both DataFrames
print(cleaned_df[['latitude', 'longitude']].duplicated().sum())
print(distances_data[['csv_latitude', 'csv_longitude']].duplicated().sum())


72412
81308


In [31]:
distances_data = distances_data.drop_duplicates(subset=['csv_latitude', 'csv_longitude'])

Join the dataframe with the distances data

In [34]:
distances_data = distances_data.rename(columns={
    'latitude': 'csv_latitude',
    'longitude': 'csv_longitude'
})

# Merge the additional data with cleaned_df based on latitude and longitude
merged_with_additional = pd.merge(
    cleaned_df, distances_data, how='left',
    left_on=['latitude', 'longitude'],
    right_on=['csv_latitude', 'csv_longitude']
)

# Drop the extra columns used for merging, if you don't need them
merged_with_additional = merged_with_additional.drop(columns=['csv_latitude', 'csv_longitude'])

In [36]:
# Check for duplicates in the merged DataFrame
merged_with_additional.count()

state                     151802
suburb_x                  151802
bedrooms                  151802
bathrooms                 151802
propertyTypes             151802
carspaces                 151802
date_listed               151802
latitude                  151802
longitude                 151802
price                     151802
is_furnished              151802
year                      151802
month                     151802
day                       151802
SA2_CODE21                151802
suburb_y                  151802
population                151802
distance_to_bus_stop      114642
distance_to_train_stop    114642
distance_to_tram_stop     114642
dtype: int64

In [37]:
# Display the first few rows to verify the merge
print(merged_with_additional.head())

# Optionally, check for any missing values in the new columns
print("Missing values in new columns:")
print(merged_with_additional[['distance_to_bus_stop', 'distance_to_train_stop', 'distance_to_tram_stop']].isna().sum())

  state        suburb_x  bedrooms  bathrooms        propertyTypes  carspaces  \
0   vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
1   vic       Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   
2   vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
3   vic  Brunswick East       2.0        1.0  [apartmentUnitFlat]        1.0   
4   vic  East Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   

  date_listed   latitude   longitude  price  is_furnished  year  month  day  \
0  2011-01-15 -37.815016  144.965968  380.0             0  2011      1   15   
1  2011-01-17 -37.816257  144.953288  550.0             0  2011      1   17   
2  2011-01-17 -37.821294  144.955768  370.0             0  2011      1   17   
3  2011-01-31 -37.760855  144.961691  429.0             0  2011      1   31   
4  2011-02-14 -37.810074  144.985476  460.0             0  2011      2   14   

  SA2_CODE21              suburb_y populatio

In [38]:
# Drop rows with missing values in any of the distance columns
distance_columns = ['distance_to_bus_stop', 'distance_to_train_stop', 'distance_to_tram_stop']
cleaned_with_distances = merged_with_additional.dropna(subset=distance_columns)

# Display the shape of the DataFrame before and after dropping rows
print(f"Original DataFrame shape: {merged_with_additional.shape}")
print(f"DataFrame shape after dropping rows with missing distance values: {cleaned_with_distances.shape}")

# Optionally, reset the index
cleaned_with_distances.reset_index(drop=True, inplace=True)

# Display the first few rows to verify
print(cleaned_with_distances.head())

Original DataFrame shape: (151802, 20)
DataFrame shape after dropping rows with missing distance values: (114642, 20)
  state        suburb_x  bedrooms  bathrooms        propertyTypes  carspaces  \
0   vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
1   vic       Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   
2   vic       Melbourne       1.0        1.0  [apartmentUnitFlat]        0.0   
3   vic  Brunswick East       2.0        1.0  [apartmentUnitFlat]        1.0   
4   vic  East Melbourne       2.0        1.0  [apartmentUnitFlat]        1.0   

  date_listed   latitude   longitude  price  is_furnished  year  month  day  \
0  2011-01-15 -37.815016  144.965968  380.0             0  2011      1   15   
1  2011-01-17 -37.816257  144.953288  550.0             0  2011      1   17   
2  2011-01-17 -37.821294  144.955768  370.0             0  2011      1   17   
3  2011-01-31 -37.760855  144.961691  429.0             0  2011      1   31   
4  201

In [39]:
cleaned_with_distances.count()

state                     114642
suburb_x                  114642
bedrooms                  114642
bathrooms                 114642
propertyTypes             114642
carspaces                 114642
date_listed               114642
latitude                  114642
longitude                 114642
price                     114642
is_furnished              114642
year                      114642
month                     114642
day                       114642
SA2_CODE21                114642
suburb_y                  114642
population                114642
distance_to_bus_stop      114642
distance_to_train_stop    114642
distance_to_tram_stop     114642
dtype: int64

In [40]:
model_df = cleaned_with_distances.copy()

Engineer features that will assist with modelling

In [41]:
# Example: Interaction between number of bedrooms and bathrooms
model_df['bed_bath_interaction'] = model_df['bedrooms'] * model_df['bathrooms']

In [44]:
# Create a new column for the reference date
reference_date = pd.Timestamp('2011-01-01')

# Create a new column 'date_numeric' to store the number of days since the reference date
model_df['date_numeric'] = pd.to_datetime(model_df[['year', 'month', 'day']]) - reference_date
model_df['date_numeric'] = model_df['date_numeric'].dt.days

# Drop the original year, month, and day columns if no longer needed
model_df = model_df.drop(columns=['month', 'day'])

In [45]:
print(model_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114642 entries, 0 to 114641
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   bedrooms                114642 non-null  float64       
 1   bathrooms               114642 non-null  float64       
 2   propertyTypes           114642 non-null  object        
 3   carspaces               114642 non-null  float64       
 4   date_listed             114642 non-null  datetime64[ns]
 5   latitude                114642 non-null  float64       
 6   longitude               114642 non-null  float64       
 7   price                   114642 non-null  float64       
 8   is_furnished            114642 non-null  int64         
 9   year                    114642 non-null  int32         
 10  SA2_CODE21              114642 non-null  object        
 11  suburb_y                114642 non-null  object        
 12  population              114642

In [53]:
model_df = model_df.drop(columns=['date_listed', 'suburb_y', 'suburb_x', 'state'])

In [55]:
# Save the DataFrame to a Parquet file
output_file = '../data/curated/model_data.parquet'
model_df.to_parquet(output_file, index=False)
